# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [73]:
import pandas as pd
import requests


sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape



(27028, 69)


(27028, 69)

In [18]:
# Valores Missing

sinasc.isna().sum()


ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [31]:
# 3) colunas selecionadas

sinasc_duo = sinasc.loc[:,['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 
              'QTDFILVIVO','GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isna().sum()
sinasc_duo


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [46]:
# 4) número de linhas e o número de missings APGAR5

sinasc.drop_duplicates(subset=['APGAR5'])

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
14,1,2679477.0,110001,1,31,1.0,4 a 7 anos,999992.0,1.0,0.0,...,5,15,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
20,1,NaN,110001,4,29,5.0,4 a 7 anos,621005.0,3.0,0.0,...,4,21,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
31,1,2679477.0,110001,1,21,2.0,8 a 11 anos,621005.0,0.0,0.0,...,5,32,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
182,1,2494299.0,110002,1,30,5.0,8 a 11 anos,999992.0,2.0,0.0,...,3,185,ATIVO,MUNIC,Ariquemes,Rondônia,-9.90846,-63.03327,139.0,4426.571
185,1,2494299.0,110002,1,24,1.0,8 a 11 anos,999992.0,0.0,1.0,...,3,188,ATIVO,MUNIC,Ariquemes,Rondônia,-9.90846,-63.03327,139.0,4426.571
205,1,2494299.0,110002,1,28,2.0,4 a 7 anos,999992.0,2.0,0.0,...,4,208,ATIVO,MUNIC,Cujubim,Rondônia,-9.37819,-62.58717,111.0,3863.943
418,1,2494299.0,110002,1,14,5.0,4 a 7 anos,999992.0,0.0,0.0,...,4,422,ATIVO,MUNIC,Monte Negro,Rondônia,-10.26310,-63.29556,155.0,1931.378
558,1,2515504.0,110002,1,30,1.0,12 anos ou mais,999992.0,0.0,0.0,...,5,562,ATIVO,MUNIC,Buritis,Rondônia,-10.20981,-63.82968,153.0,3265.809


In [51]:
# Missing APGAR 5
sinasc['APGAR5'].isna().sum()

103

In [53]:
# 5) Código 9
ignorado = sinasc.loc[:,['ESTCIVMAE', 'CONSULTAS',]]
ignorado

,ESTCIVMAE,CONSULTAS
0,5.0,4
1,2.0,4
2,9.0,4
3,5.0,3
4,2.0,4
...,...,...
27023,2.0,4
27024,1.0,4
27025,2.0,4
27026,2.0,4


In [60]:
# 6) substituir por 0 zero
sinasc.loc[:,['QTDFILVIVO']].fillna(0)

,QTDFILVIVO
0,0.0
1,1.0
2,2.0
3,0.0
4,1.0
...,...
27023,1.0
27024,0.0
27025,0.0
27026,1.0


In [64]:
# 8) seu código aqui
sinasc_apgar5 = sinasc.loc[:,['APGAR5']]
sinasc_apgar5.loc[sinasc_apgar5['APGAR5'] <= 3, 'APGAR5_R'] = 'asfixia_severa'
sinasc_apgar5.loc[(sinasc_apgar5['APGAR5'] > 3) & (sinasc_apgar5['APGAR5'] <= 5),
           'APGAR5_R'] = 'asfixia_moderada'
sinasc_apgar5.loc[(sinasc_apgar5['APGAR5'] > 5) & (sinasc_apgar5['APGAR5'] <= 7),
           'APGAR5_R'] = 'asfixia_leve'
sinasc_apgar5.loc[(sinasc_apgar5['APGAR5'] > 7), 'APGAR5_R'] = 'normal'
sinasc_apgar5

,APGAR5,APGAR5_R
0,10.0,normal
1,9.0,normal
2,10.0,normal
3,10.0,normal
4,10.0,normal
...,...,...
27023,9.0,normal
27024,9.0,normal
27025,10.0,normal
27026,9.0,normal


In [65]:
# 9) seu código aqui
sinasc_apgar5['APGAR5'].sort_values()
sinasc_apgar5['APGAR5'].quantile([0.25,0.5,0.75])

0.25     9.0
0.50     9.0
0.75    10.0
Name: APGAR5, dtype: float64

In [77]:
import numpy as np
sinasc_apgar5['imc_cat_3'] = np.where(
    sinasc_apgar5['APGAR5'] < 9, '1o_quartil',
    np.where(sinasc_apgar5['APGAR5'] < 9, '2o_quartil',
             np.where(sinasc_apgar5['APGAR5'] < 10, '3o_quartil', '4o_quartil')))
sinasc_apgar5

,APGAR5,APGAR5_R,imc_cat_3
0,10.0,normal,4o_quartil
1,9.0,normal,3o_quartil
2,10.0,normal,4o_quartil
3,10.0,normal,4o_quartil
4,10.0,normal,4o_quartil
...,...,...,...
27023,9.0,normal,3o_quartil
27024,9.0,normal,3o_quartil
27025,10.0,normal,4o_quartil
27026,9.0,normal,3o_quartil
